# Code example 0c: ESMFold

<a href="https://colab.research.google.com/github/BioGeMT/MALTAomics-Summer-School/blob/main/Day4_WorkshopVII_DeepLearningForProteinStructure/maltaomics_ex0c_esmfold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Downloading an already existing structure from [AlphaFold database](https://alphafold.ebi.ac.uk/ ):

In [1]:
#############################################################################
# TODO: Find a UniProt ID of some protein that has a structure available in #
# the AlphaFold database: https://alphafold.ebi.ac.uk/                      #
#############################################################################

UNIPROT_ID = 'O15552' #'P31456' #'A0A3B3QWR5'
PATH = '/content/'

In [2]:
import requests

def download_af_structure(uniprot_id):
  try:
    # download 3D structure based on UniProt ID:
    response = requests.get(f'https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb')
    if not response.ok or response.text == 'N/A':
      print(f'AF structure not found for {uniprot_id}.')
    structure = response.text

    # save the downloaded structure to a PDB file:
    structure_path = f'{PATH}/af-{UNIPROT_ID}.pdb'
    with open(structure_path, 'w') as f:
      f.write(structure)
    return structure_path

  except:
    print(f'Exception {uniprot_id}')
  return ''

structure_path = download_af_structure(UNIPROT_ID)
structure_path

'/content//af-O15552.pdb'

## 2. Predicting a structure from an amino acid sequence:

Using **OmegaFold** with default parameters to predict the structure:

(Simplified from https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/ESMFold.ipynb#scrollTo=CcyNpAvhTX6q, takes ~5 minutes.)

In [3]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.1 MB/s eta 0:00:00


We will extract the amino acid sequence from a PDB file:

In [4]:
from Bio import SeqIO

def get_aa_seq(pdb_path):
  # (more code because SeqIO.parse returns a generator in case the input file
  # contains more than one records)
  aa_seq = []
  for record in SeqIO.parse(pdb_path, 'pdb-atom'):
    aa_seq.append(record.seq)

  if len(aa_seq) == 1:
    aa_seq = str(aa_seq[0])

  return aa_seq

sequence = get_aa_seq(structure_path)
sequence

/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(


'MLPDWKSSLILMAYIIIFLTGLPANLLALRAFVGRIRQPQPAPVHILLLSLTLADLLLLLLLPFKIIEAASNFRWYLPKVVCALTSFGFYSSIYCSTWLLAGISIERYLGVAFPVQYKLSRRPLYGVIAALVAWVMSFGHCTIVIIVQYLNTTEQVRSGNEITCYENFTDNQLDVVLPVRLELCLVLFFIPMAVTIFCYWRFVWIMLSQPLVGAQRRRRAVGLAVVTLLNFLVCFGPYNVSHLVGYHQRKSPWWRSIAVVFSSLNASLDPLLFYFSSSVVRRAFGRGLQVLRNQGSSLLGRRGKDTAEGTNEDRGVGQGEGMPSSDFTTE'

In [5]:
import os, time

model_name = 'esmfold.model'

os.system('apt-get install aria2 -qq')
os.system(f'aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/{model_name} &')

# install libs
os.system('pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol')

# install openfold
os.system(f'pip install -q git+https://github.com/aqlaboratory/openfold.git@6908936b68ae89f67755240e2f588c09ec31d4c8')

# install esmfold
os.system(f'pip install -q git+https://github.com/sokrypton/esm.git')

# wait for parameters to finish downloading...
while not os.path.isfile(model_name):
  time.sleep(5)
if os.path.isfile(f'{model_name}.aria2'):
  print('Downloading params...')
while os.path.isfile(f'{model_name}.aria2'):
  time.sleep(5)

In [6]:
predicted_structure_path = f'{PATH}/{UNIPROT_ID}.pdb'
predicted_structure_path

'/content//O15552.pdb'

Running ESMFold:

In [7]:
import torch
from jax.tree_util import tree_map

model = torch.load(model_name)
model.eval().cuda().requires_grad_(False)
model_name_ = model_name

torch.cuda.empty_cache()
output = model.infer(sequence)

pdb_str = model.output_to_pdb(output)[0]
output = tree_map(lambda x: x.cpu().numpy(), output)
plddt = output['plddt'][0,...,1].mean()
print(f'plddt: {plddt:.3f}')

with open(predicted_structure_path,'w') as out:
  out.write(pdb_str)

plddt: 87.768


Display the predicted structure:

In [8]:
import py3Dmol

# https://william-dawson.github.io/using-py3dmol.html
def display_structure(pdb_path):
  view = py3Dmol.view()
  view.addModel(open(pdb_path, 'r').read(),'pdb')

  # colour by pLDDT:
  view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})

  view.zoomTo()
  view.show()

display_structure(predicted_structure_path)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 3. Compare the two structures - align them minimizing RMSD:

Source: https://gist.github.com/andersx/6354971

**Root Mean Square Deviation** ([RMSD](https://en.wikipedia.org/wiki/Root-mean-square_deviation_of_atomic_positions))

== measure of the average distance between the atoms (usually the backbone atoms) of superimposed proteins


$$ RMSD = \sqrt{{{1} \over {N}} \sum_{i=1}^{N} \delta_i^2} $$

In [9]:
import Bio.PDB

def get_atoms_to_be_aligned(some_structure, atoms_range):
  atoms = []
  for chain in some_structure:
    # to find proper atoms iterate over all residues (amino acids) in the chain
    for residue in chain:
      if residue.get_id()[1] in atoms_range:
        atoms.append(residue['CA'])  # append CA atom to list -- C alpha is the
                                     # position of the functional group (what
                                     # responsible for the molecule's chemical
                                     # characteristics)
  return atoms

def align_two_structures(reference_structure_path, sample_structure_path):
  ref_str_len = len(get_aa_seq(reference_structure_path))
  sample_str_len = len(get_aa_seq(sample_structure_path))
  max_str_len = min(ref_str_len, sample_str_len)

  atoms_to_be_aligned = range(1, max_str_len)

  pdb_parser = Bio.PDB.PDBParser()
  ref_structure = pdb_parser.get_structure('reference', reference_structure_path)[0]
  sample_structure = pdb_parser.get_structure('sample', sample_structure_path)[0]

  # make a list of the atoms (in the structures) you wish to align
  # (here we use CA atoms whose index is in the specified range)
  ref_atoms = get_atoms_to_be_aligned(ref_structure, atoms_to_be_aligned)
  sample_atoms = get_atoms_to_be_aligned(sample_structure, atoms_to_be_aligned)

  # initiate the superimposer
  super_imposer = Bio.PDB.Superimposer()
  super_imposer.set_atoms(ref_atoms, sample_atoms)
  super_imposer.apply(sample_structure.get_atoms())

  # RMSD
  print(super_imposer.rms)

  # save the aligned version of the sample structure
  io = Bio.PDB.PDBIO()
  io.set_structure(sample_structure)
  aligned_structure_path = f'{PATH}/aligned.pdb'
  io.save(aligned_structure_path)
  return aligned_structure_path

structure_path = f'{PATH}/af-{UNIPROT_ID}.pdb'
aligned_structure_path = align_two_structures(structure_path, predicted_structure_path)
aligned_structure_path

/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2626
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(


10.166552667232919


/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2626
  warnings.warn(


'/content//aligned.pdb'

Display the AlphaFold (orange) and predicted (blue) structures before alignment:

In [10]:
def display_two_structures(pdb_path_reference, pdb_path_sample):
  view = py3Dmol.view()
  view.addModel(open(pdb_path_reference, 'r').read(), 'pdb')
  view.addModel(open(pdb_path_sample, 'r').read(), 'pdb')
  view.setStyle({'cartoon': {'color': 'orange'}})
  # (the predicted structure is blue, https://william-dawson.github.io/using-py3dmol.html)
  view.setStyle({'model': -1}, {'cartoon': {'color': 'blue'}})
  view.zoomTo()
  view.show()

display_two_structures(structure_path, predicted_structure_path)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Display the AlphaFold (orange) and predicted (blue) structures after alignment:

In [11]:
display_two_structures(structure_path, aligned_structure_path)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Are we getting the same results for our `amino acid sequence -> 3D structure` prediction as what is in the AF2 database - why?